### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Initialise portfolio （Go back here everytime you want to restart.）

In [17]:
;; initialise with current date and initial capital (= $100000)
(init-portfolio "1980-12-31" 100000);

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 50 cross above the MA 200 (golden cross)

MA 200 cross below the MA 50 (death cross)

So in the codes, MA50 and MA200 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



In [18]:
(def MA50-vec-aapl [])
(def MA200-vec-aapl [])
(def MA50-vec-f [])
(def MA200-vec-f [])
(while (not= (get-date) "2016-12-29")
    (do
      (def tics (deref available-tics)) ;20 ms
      (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl))
      (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl))
      (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f ))
      (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
        (if (> MA50 MA200)
          (order "AAPL" 1 :print false) 
          (order "AAPL" 0 :remaining true))))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
        (if (> MA50 MA200)
          (order "F" 1  :print false) 
          (order "F" 0 :remaining true )))
      (next-date))
(end-order)
;(update-eval-report (get-date))

nil

In [19]:
(pprint/print-table (take 200 (deref portfolio-value)))


|      :date |         :tot-value | :daily-ret |               :tot-ret | :loan | :leverage |
|------------+--------------------+------------+------------------------+-------+-----------|
| 1980-12-31 |             100000 |        0.0 |                    0.0 |   0.0 |       0.0 |
| 1981-01-02 |           100000.0 |        0.0 |                    0.0 |   0.0 |       0.0 |
| 1981-01-05 |           100000.0 |        0.0 |                    0.0 |   0.0 |       0.0 |
| 1981-01-06 |           100000.0 |        0.0 |                    0.0 |   0.0 |       0.0 |
| 1981-01-07 |           100000.0 |        0.0 |                    0.0 |   0.0 |       0.0 |
| 1981-01-08 |           100000.0 |        0.0 |                    0.0 |   0.0 |       0.0 |
| 1981-01-09 |           100000.0 |        0.0 |                    0.0 |   0.0 |       0.0 |
| 1981-01-12 |           100000.0 |        0.0 |                    0.0 |   0.0 |       0.0 |
| 1981-01-13 |           100000.0 |        0.0 |           

| 1981-05-05 |  99926.65322559219 |        0.0 |  -3.186578708329458E-4 |   0.0 |       0.0 |
| 1981-05-06 |  99950.44373728606 |        0.0 | -2.1527345955130552E-4 |   0.0 |       0.0 |
| 1981-05-07 |  99990.93160275441 |        0.0 |  -3.938533467081503E-5 |   0.0 |       0.0 |
| 1981-05-08 |  99999.21896673653 |        0.0 | -3.3919976113061946E-6 |   0.0 |       0.0 |
| 1981-05-11 |  99938.75612007371 |        0.0 | -2.6606027215874957E-4 |   0.0 |       0.0 |
| 1981-05-12 |   99965.6513990338 |        0.0 | -1.4919970407969204E-4 |   0.0 |       0.0 |
| 1981-05-13 |  99974.82784501972 |        0.0 | -1.0933504162574983E-4 |   0.0 |       0.0 |
| 1981-05-14 | 100021.61231717377 |        0.0 |   9.385095957898766E-5 |   0.0 |       0.0 |
| 1981-05-15 | 100097.06741093971 |        0.0 |  4.2135394382305824E-4 |   0.0 |       0.0 |
| 1981-05-18 | 100077.85658180484 |        0.0 |    3.37995279857943E-4 |   0.0 |       0.0 |
| 1981-05-19 | 100077.85658180484 |        0.0 |    3.379952

| 1981-09-09 |   99450.4807274392 |        0.0 | -0.0023931132256230082 |   0.0 |       0.0 |
| 1981-09-10 |   99450.4807274392 |        0.0 | -0.0023931132256230082 |   0.0 |       0.0 |
| 1981-09-11 |   99450.4807274392 |        0.0 | -0.0023931132256230082 |   0.0 |       0.0 |
| 1981-09-14 |   99450.4807274392 |        0.0 | -0.0023931132256230082 |   0.0 |       0.0 |
| 1981-09-15 |   99450.4807274392 |        0.0 | -0.0023931132256230082 |   0.0 |       0.0 |
| 1981-09-16 |   99450.4807274392 |        0.0 | -0.0023931132256230082 |   0.0 |       0.0 |
| 1981-09-17 |   99450.4807274392 |        0.0 | -0.0023931132256230082 |   0.0 |       0.0 |
| 1981-09-18 |   99450.4807274392 |        0.0 | -0.0023931132256230082 |   0.0 |       0.0 |
| 1981-09-21 |   99450.4807274392 |        0.0 | -0.0023931132256230082 |   0.0 |       0.0 |
| 1981-09-22 |   99450.4807274392 |        0.0 | -0.0023931132256230082 |   0.0 |       0.0 |
| 1981-09-23 |   99450.4807274392 |        0.0 | -0.00239311

nil

### Check portfolio record

In [20]:
;; view final portfolio
(view-portfolio)


| :asset | :price |  :aprc | :quantity | :tot-val |
|--------+--------+--------+-----------+----------|
|   cash |    N/A |    N/A |       N/A |   195145 |
|   AAPL | 116.73 | 312.11 |        81 |    25280 |
|      F |  12.23 | 577.33 |         0 |        0 |


nil

In [ ]:
;; view portfolio value and return
(view-portfolio-record)

### Generate evaluation report

In [8]:
(eval-report)

nil

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)